In [1]:
import os
import torch
import re
import copy
import torch.nn as nn
from torch.nn import Embedding,Conv1d
from pathlib import Path

import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn,optim
from tqdm.notebook import tqdm

from torch.optim import Adam



import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

# LSTM and CNN Work

In [3]:
root_path = Path("../input/covid-19-nlp-text-classification")  
train_df = pd.read_csv(root_path / "Corona_NLP_train.csv",encoding='latin')  
test_df = pd.read_csv(root_path / "Corona_NLP_test.csv",encoding='latin')
train_df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
embedding_dim = 300  # 设定需要的embedding长度
# word_dict是一个单词于id的映射字典，
# <pading>的意思是补0，<unk>代表不认识的单词，其实就是glove词向量中没有的单词都会被认为是<unk>
word_dict = {'<pading>':0,"<unk>": 1}


# 加载对应长度的glove预训练词向量，维度越大的词向量加载越慢，300维的词向量文件有1G
glove_path = Path("/kaggle/input/glove6b")  
glove_df = pd.read_csv(glove_path / f"glove.6B.{embedding_dim}d.txt", sep=" ", quoting=3, header=None, index_col=0)
# 生成对应的字典形式，key为单词，value为词向量
glove_dict = {key: val.values for key, val in glove_df.T.items()}

In [5]:
# code from https://www.kaggle.com/code/tharunikag/corona-virus-tweets-classification
# Cleaning the text
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

def preprocessor(data):
    corpus = []
    for i in range(len(data)):
        #remove urls
        tweet1= re.sub(r'http\S+', ' ', data[i])
        #remove html tags
        tweet2 = re.sub(r'<.*?>',' ', tweet1) 
        #remove digits
        tweet3 = re.sub(r'\d+',' ', tweet2)
        #remove hashtags
        tweet4 = re.sub(r'#\w+',' ', tweet3)
        review = re.sub('[^a-zA-Z]', ' ', tweet4)
        review = review.lower()
        review = review.split()
        # ps = PorterStemmer()
        # review = [ps.stem(word) for word in review if word not in all_stopwords]
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in all_stopwords]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

#train_df  = preprocessor(train_df)
#test_df  = preprocessor(test_df)
#train_df.head()

In [8]:
def word_tokenize(text: str):
    """ 
    这是一个切分单词的函数，这个函数除了简单了分词之外，
    还会将word_dict补充完整，生成完整的词表映射
    """
    word_index = []
    pat = re.compile(r"[\w]+|[.,!?;|]") 
    tokens = pat.findall(text.lower())  
    for token in tokens:
        if token not in word_dict:
            word_dict[token] = len(word_dict) if token in glove_dict else word_dict["<unk>"]
        word_index.append(word_dict[token])
    return word_index
# 训练集和测试集分词
MAX_LENGTH = 240  # 最大句子长度

train_df["OriginalTweet"] = preprocessor(train_df["OriginalTweet"])
test_df["OriginalTweet"] = preprocessor(test_df["OriginalTweet"])
train_text = train_df["OriginalTweet"].apply(lambda s: word_tokenize(str(s)))
test_text = test_df["OriginalTweet"].apply(lambda s: word_tokenize(str(s)))
train_text = train_text.apply(lambda x:(x+[0]*MAX_LENGTH)[:MAX_LENGTH])
test_text = test_text.apply(lambda x:(x+[0]*MAX_LENGTH)[:MAX_LENGTH])

MAX_LENGTH = 240  # 最大句子长度

s2i = {'Positive':1,'Extremely Positive':0,'Neutral':2,'Extremely Negative':4,'Negative':3}
train_df['Sentiment']=train_df['Sentiment'].replace(s2i).astype(int)
test_df['Sentiment']=test_df['Sentiment'].replace(s2i).astype(int)


In [9]:
train_df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,menyrbie phil gahan chrisitv,2
1,3800,48752,UK,16-03-2020,advice talk neighbour family exchange phone nu...,1
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworth give elderly d...,1
3,3802,48754,NaN,16-03-2020,food stock not one empty please panic enough f...,1
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak not paranoid foo...,4


In [10]:
# 预训练的词向量矩阵，这个后面会直接输入embedding层
glove_embeddings = np.zeros((len(word_dict), embedding_dim))
for k, v in word_dict.items():
    if v==0:
        glove_embeddings[v] = np.zeros(embedding_dim)
    glove_embeddings[v] = glove_dict[k] if k in glove_dict else glove_dict["<unk>"]

## 数据集构建

In [11]:
class CommentDataset(Dataset):
    def __init__(self,texts,labels):
        self.texts=texts
        self.labels=labels
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,item):
        """
        item 为数据索引，迭代取第item条数据
        """
        text=torch.tensor(self.texts[item],dtype=torch.long)
        label=torch.tensor(self.labels[item],dtype=torch.long)
        return {'text_id':text,'label':label}
def create_data_loader(X,y,batch_size):
    ds=CommentDataset(
        texts = X.values,
        labels=y.values
    )
    return DataLoader(
        ds,
        batch_size=batch_size
    )
BATCH_SIZE = 128
train_data_loader = create_data_loader(train_text,train_df['Sentiment'], BATCH_SIZE)
val_data_loader = create_data_loader(test_text,test_df['Sentiment'], BATCH_SIZE)

# 模型构建

In [12]:
class TextCNN(nn.Module): 
    def __init__(self,weight, embedding_dim=300,dropout=0.4):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(weight,freeze=False)  # 仍然训练词向量
        self.kernel_size = [3,4,5,6]
        self.conv = nn.ModuleList([nn.Sequential(
                Conv1d(in_channels=embedding_dim, out_channels=1, kernel_size=k, stride=2),
                nn.PReLU(),
                nn.MaxPool1d(4, stride=2)) for k in self.kernel_size])
        self.Linear = nn.Linear(232,5)
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, X):
        X = self.embedding(X).permute(0,2,1)    # [b,240] ==> [b,240,300] ==> [b,300,240]
        X = [c(X).squeeze() for c in self.conv]  # [b,300,240] ==> [b,58] * 4
        X = torch.cat(X,dim=1)  # [b,232]
        out = self.dropout(self.Linear(X)) # [b,300] ==> [b,5]
        return out

## 模型训练

In [13]:
from collections import defaultdict
history = defaultdict(list)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 设置CUDA
N_EPOCHS = 15     # 设置模型训练次数
learning_rate = 4e-4  # 初始学习率
model_name = 'TextCNN'
model= TextCNN(torch.from_numpy(glove_embeddings).float())
model = model.to(device)
# 优化器
optimizer = Adam(model.parameters(),learning_rate)

# 损失
criterion = nn.CrossEntropyLoss().to(device)
best_acc = 0
for epoch in tqdm(range(N_EPOCHS)):
    #启用模型的训练模式
    model.train()
    # 定义损失
    epoch_loss = 0
    epoch_acc = 0
    val_number = 0
    for batch in train_data_loader:
        text_id = batch['text_id'].to(device)
        label = batch['label'].to(device)
        predictions = model(text_id)
        loss = criterion(predictions, label)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        preds = predictions.max(1)[1]
        epoch_acc += ((preds==label).sum().item())
        val_number += label.size(0)
    history[f'{model_name}_train_loss'].append(epoch_loss / len(train_data_loader))
    history[f'{model_name}_train_accuracy'].append(epoch_acc / val_number)
    print(f'epoch {epoch+1}, Training Loss：',epoch_loss / len(train_data_loader),', accuracy: ',epoch_acc / val_number)
    
    model.eval()
    # 初始化损失
    epoch_loss = 0
    epoch_acc = 0
    val_number = 0

    #不计算梯度
    with torch.no_grad():
        for batch in val_data_loader:
            text_id = batch['text_id'].to(device)
            label = batch['label'].to(device)
            predictions = model(text_id)
            loss = criterion(predictions, label)
            epoch_loss += loss.item()
            preds = predictions.max(1)[1]
            epoch_acc += ((preds==label).sum().item())
            val_number += label.size(0)
    history[f'{model_name}_test_loss'].append(epoch_loss / len(val_data_loader))
    history[f'{model_name}_test_accuracy'].append(epoch_acc / val_number)
    if best_acc < (epoch_acc / val_number):
        best_acc = epoch_acc / val_number
    print(f'epoch {epoch+1}, Testing Loss：',epoch_loss / len(val_data_loader),', accuracy:',epoch_acc / val_number)
    print('-'*20)
print(model_name,'best Accuracy:',best_acc)

  0%|          | 0/15 [00:00<?, ?it/s]

epoch 1, Training Loss： 1.5299768336811421 , accuracy:  0.2972033918895935
epoch 1, Testing Loss： 1.4257853706677754 , accuracy: 0.3651922064244339
--------------------
epoch 2, Training Loss： 1.372280348161733 , accuracy:  0.38904682071093616
epoch 2, Testing Loss： 1.2176870107650757 , accuracy: 0.5152711953659821
--------------------
epoch 3, Training Loss： 1.2398337325694397 , accuracy:  0.4615253784289428
epoch 3, Testing Loss： 1.0824696600437165 , accuracy: 0.580568720379147
--------------------
epoch 4, Training Loss： 1.1537511258021644 , accuracy:  0.5083946837718979
epoch 4, Testing Loss： 1.007916933298111 , accuracy: 0.6248025276461295
--------------------
epoch 5, Training Loss： 1.107594781040405 , accuracy:  0.5339310445367739
epoch 5, Testing Loss： 0.9664780775705973 , accuracy: 0.6400737230121116
--------------------
epoch 6, Training Loss： 1.0691965190890413 , accuracy:  0.5531258352163666
epoch 6, Testing Loss： 0.9421229322751363 , accuracy: 0.6490258030542391
----------

In [ ]:
len(history['TextCNN_test_accuracy'][10:]),len(history['LSTM_test_accuracy'])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.style.use('ggplot')
plt.figure(figsize=(16,5),dpi=150)
plt.subplot(1,2,1)
plt.plot(history['LSTM_test_loss'][1:],label='LSTM')
plt.plot(history['TextCNN_test_loss'][10:],label='TextCNN')
# plt.plot(history['CNNText_inception_test_loss'],label='CNNText_inception')
plt.title('Test Loss')
plt.legend(frameon=True,loc='lower right',facecolor='white')
plt.xticks(range(1,21))
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(4)) # 横坐标间隔为5

plt.subplot(1,2,2)
plt.plot(history['LSTM_test_accuracy'][1:],label='LSTM')
plt.plot(history['TextCNN_test_accuracy'][10:],label='TextCNN')
# plt.plot(history['CNNText_inception_test_accuracy'],label='CNNText_inception')
plt.title('Test Accuracy')
plt.legend(frameon=True,loc='lower right',facecolor='white')
plt.xticks(range(1,21))
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(4)) # 横坐标间隔为5

fig.show()

In [ ]:
# history = defaultdict(list,
#             {'LSTM_train_loss': [1.308557040943122,
#               1.278768042970148,
#               0.7531606548506281,
#               0.5597835259500498,
#               0.4386700326056214,
#               0.3499181439047274,
#               0.3045885660309599,
#               0.26094121992680597,
#               0.2147264429917617,
#               0.1877273618654342,
#               0.152648972567194,
#               0.13610333766533722,
#               0.13214566704950162,
#               0.12991164165392796,
#               0.09901983114480047,
#               0.07889837335205134,
#               0.0704744360163831,
#               0.06673340640979478,
#               0.06895074986430429,
#               0.05591371395662774,
#               0.050204002620906844],
#              'LSTM_train_accuracy': [0.41725587384892,
#               0.4352843987657021,
#               0.7177393881964186,
#               0.8024151420171538,
#               0.8498675802415142,
#               0.8810165949899167,
#               0.8957892946521855,
#               0.910659183128022,
#               0.9272055786378988,
#               0.9364628131302087,
#               0.9498748694025317,
#               0.9552202541487474,
#               0.956216439487815,
#               0.9567995723692203,
#               0.9670772894039896,
#               0.9741963699978132,
#               0.9778652477099886,
#               0.9784483805913939,
#               0.9780353281337318,
#               0.9814126394052045,
#               0.9842554122020556],
#              'LSTM_test_loss': [0.9859559595584869,
#               0.9611425340175629,
#               0.7729674458503724,
#               0.6429263601700465,
#               0.6459331264098486,
#               0.7000426411628723,
#               0.66990860303243,
#               0.7279503504435222,
#               0.7658875823020935,
#               0.7978982090950012,
#               0.8314582268397014,
#               0.9488745947678884,
#               0.820837511618932,
#               0.880662182966868,
#               0.9553210179011027,
#               0.9655674556891124,
#               1.0165971755981444,
#               1.2468677679697673,
#               1.2663498044013977,
#               1.4162139733632406,
#               1.2338464121023813],
#              'LSTM_test_accuracy': [0.6134807793575566,
#               0.6327014218009479,
#               0.7211690363349131,
#               0.775144813059505,
#               0.7688256977356503,
#               0.7677725118483413,
#               0.7759347024749869,
#               0.7735650342285414,
#               0.7830437072143234,
#               0.779884149552396,
#               0.7854133754607688,
#               0.7598736176935229,
#               0.7833070036861506,
#               0.7711953659820958,
#               0.7654028436018957,
#               0.7740916271721959,
#               0.7785676671932596,
#               0.7682991047919958,
#               0.7746182201158505,
#               0.779884149552396,
#               0.7906793048973144],
#              'TextCNN_train_loss': [1.5541518723002132,
#               1.508095990426792,
#               1.4175605555498823,
#               1.3293673658963316,
#               1.262729629596568,
#               1.21605933860222,
#               1.175219433463138,
#               1.1382055710191312,
#               1.102624380625553,
#               1.0745478268735897,
#               1.5580135887454014,
#               1.4740827427887768,
#               1.3625942546388377,
#               1.2586467384551623,
#               1.1741743139598682,
#               1.1173091962100556,
#               1.0864043829974181,
#               1.0605159233445707,
#               1.0313131785170633,
#               1.0188545529146371,
#               0.9947913492688482,
#               0.9795045700872907,
#               0.9698766234868802,
#               0.9537591410349615,
#               0.9300512714045388,
#               0.915457517649076,
#               0.8966342862348379,
#               0.8825370086276013,
#               0.8693738192875192,
#               0.8511631664652262],
#              'TextCNN_train_accuracy': [0.28116723765094637,
#               0.3150375391792405,
#               0.3730835580824647,
#               0.4219695313069466,
#               0.4604320042763078,
#               0.4844862356342785,
#               0.5056005053818305,
#               0.5237991107223559,
#               0.5405641810627597,
#               0.5522754330976505,
#               0.27633209417596033,
#               0.3349126515538062,
#               0.3922540515586656,
#               0.44791894452948466,
#               0.50044949826275,
#               0.5322545375027334,
#               0.5487037441990427,
#               0.5619457200476226,
#               0.5738027553028646,
#               0.5821366960662828,
#               0.592657385134971,
#               0.5994120076779162,
#               0.6009427314916053,
#               0.6070413295429696,
#               0.6188983647982117,
#               0.6210851131034818,
#               0.6286658405617513,
#               0.6341570085283184,
#               0.6356391379352236,
#               0.6439730786986417],
#              'TextCNN_test_loss': [1.5163259784380594,
#               1.4453606406847637,
#               1.335492523511251,
#               1.2549929976463319,
#               1.2098475058873495,
#               1.1543169260025024,
#               1.1254674990971882,
#               1.0871187965075175,
#               1.0715699334939321,
#               1.0470512529214224,
#               1.5057435433069866,
#               1.3705288847287496,
#               1.2524182478586832,
#               1.1231730024019877,
#               1.0351792951424916,
#               0.9841812630494435,
#               0.9515746573607127,
#               0.9296340823173523,
#               0.9124479015668233,
#               0.9023188153902689,
#               0.8958919266859691,
#               0.8941029489040375,
#               0.8945300658543904,
#               0.8933592100938161,
#               0.8923281292120616,
#               0.8959562043348949,
#               0.8979606866836548,
#               0.905497803290685,
#               0.9122726003328959,
#               0.9189714709917705],
#              'TextCNN_test_accuracy': [0.30068457082675093,
#               0.36334913112164297,
#               0.43812532912058977,
#               0.4934175882043181,
#               0.4973670352817272,
#               0.5347551342812006,
#               0.5534491837809373,
#               0.5621379673512374,
#               0.5758293838862559,
#               0.5913638757240653,
#               0.3233280674038968,
#               0.4081095313322802,
#               0.4921011058451817,
#               0.5655608214849921,
#               0.6321748288572933,
#               0.6521853607161664,
#               0.6513954713006845,
#               0.6724591890468667,
#               0.6695629278567667,
#               0.6769352290679305,
#               0.6795681937862033,
#               0.674565560821485,
#               0.6708794102159031,
#               0.6753554502369669,
#               0.6721958925750395,
#               0.6666666666666666,
#               0.6598209583991574,
#               0.6592943654555029,
#               0.6627172195892576,
#               0.660347551342812]})

In [ ]:
from collections import OrderedDict
from torch import nn
# 这个就是多层的卷积，然后和到了一起
class Inception(nn.Module):
    def __init__(self,cin,co,relu=True,norm=True):
        super(Inception, self).__init__()
        assert(co%4==0)
        cos=[int(co/4)]*4  #[128]*4
        self.activa=nn.Sequential()
        if norm:
            self.activa.add_module('norm',nn.LayerNorm(co))
        if relu:
            self.activa.add_module('relu',nn.ReLU(True))
        self.norm2 = nn.LayerNorm(cos[1])
        self.norm3 = nn.LayerNorm(cos[2])
        
        self.branch1 =nn.Sequential(OrderedDict([
            ('conv1', nn.Conv1d(cin,cos[0], 1,stride=1)),
            ])) 
        self.branch2_1 =nn.Sequential(OrderedDict([
            ('conv1', nn.Conv1d(cin,cos[1], 1))
            ]))
        
        self.branch2_2 =nn.Sequential(OrderedDict([
            ('relu1', nn.ReLU(inplace=True)),
            ('conv3', nn.Conv1d(cos[1],cos[1], 3,stride=1,padding=1)),
            ]))
        self.branch3_1 =nn.Sequential(OrderedDict([
            ('conv1', nn.Conv1d(cin,cos[2], 3,padding=1)),
            ]))
        self.branch3_2 =nn.Sequential(OrderedDict([
            ('relu1', nn.ReLU(inplace=True)),
            ('conv3', nn.Conv1d(cos[2],cos[2], 5,stride=1,padding=2)),
            ]))
        
        self.branch4 =nn.Sequential(OrderedDict([
            ('conv3', nn.Conv1d(cin,cos[3], 3,stride=1,padding=1)),
            ]))
        
    def forward(self,x):
        branch1=self.branch1(x)
        branch2=self.branch2_2(self.norm2(self.branch2_1(x).permute(0,2,1)).permute(0,2,1))
        branch3=self.branch3_2(self.norm3(self.branch3_1(x).permute(0,2,1)).permute(0,2,1))
        branch4=self.branch4(x)
        result=self.activa(torch.cat((branch1,branch2,branch3,branch4),1).permute(0,2,1)).permute(0,2,1)
        return result
    

class CNNText_inception(nn.Module):
    def __init__(self, weight,embedding_dim=300, incept_dim=512, content_seq_len=240, 
                 linear_hidden_size=1024, num_classes=5):
        super(CNNText_inception, self).__init__()

        self.encoder = nn.Embedding.from_pretrained(weight,freeze=False)
        
        self.content_conv=nn.Sequential(
            Inception(embedding_dim,incept_dim),#(batch_size,64,opt.content_seq_len)->(batch_size,64,(opt.content_seq_len)/2)
            Inception(incept_dim,incept_dim),
            nn.MaxPool1d(content_seq_len)
        )
        self.fc = nn.Sequential(
            nn.Linear(incept_dim,linear_hidden_size),
            nn.LayerNorm(linear_hidden_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        self.fc2 = nn.Linear(linear_hidden_size,linear_hidden_size)
        self.nf = nn.Linear(linear_hidden_size,num_classes)
        
    def forward(self,content):
        content=self.encoder(content)
              
        content_out=self.content_conv(content.permute(0,2,1))
        out=content_out.view(content_out.size(0), -1)
        tmp=self.fc(out)
        tmp2=self.fc2(tmp)
        nt = tmp + tmp2    # 残差
        
        return self.nf(nt)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 设置CUDA
N_EPOCHS = 20     # 设置模型训练次数
learning_rate = 3e-4  # 初始学习率
model_name = 'CNNText_inception'
model= CNNText_inception(torch.from_numpy(glove_embeddings).float())
model = model.to(device)
# 优化器
optimizer = Adam(model.parameters(),learning_rate)

# 损失
criterion = nn.CrossEntropyLoss().to(device)
best_acc = 0
for epoch in tqdm(range(N_EPOCHS)):
    #启用模型的训练模式
    model.train()
    # 定义损失
    epoch_loss = 0
    epoch_acc = 0
    val_number = 0
    for batch in train_data_loader:
        text_id = batch['text_id'].to(device)
        label = batch['label'].to(device)
        predictions = model(text_id)
        loss = criterion(predictions, label)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        preds = predictions.max(1)[1]
        epoch_acc += ((preds==label).sum().item())
        val_number += label.size(0)
    history[f'{model_name}_train_loss'].append(epoch_loss / len(train_data_loader))
    history[f'{model_name}_train_accuracy'].append(epoch_acc / val_number)
    print(f'第{epoch+1}轮，训练Loss：',epoch_loss / len(train_data_loader),'，训练准确率：',epoch_acc / val_number)
    
    model.eval()
    # 初始化损失
    epoch_loss = 0
    epoch_acc = 0
    val_number = 0

    #不计算梯度
    with torch.no_grad():
        for batch in val_data_loader:
            text_id = batch['text_id'].to(device)
            label = batch['label'].to(device)
            predictions = model(text_id)
            loss = criterion(predictions, label)
            epoch_loss += loss.item()
            preds = predictions.max(1)[1]
            epoch_acc += ((preds==label).sum().item())
            val_number += label.size(0)
    history[f'{model_name}_test_loss'].append(epoch_loss / len(val_data_loader))
    history[f'{model_name}_test_accuracy'].append(epoch_acc / val_number)
    if best_acc < (epoch_acc / val_number):
        best_acc = epoch_acc / val_number
    print(f'第{epoch+1}轮，测试Loss：',epoch_loss / len(val_data_loader),'，测试准确率：',epoch_acc / val_number)
    print('-'*20)
print(model_name,'best Accuracy:',best_acc)

In [ ]:
plt.style.use('ggplot')  # seaborn-whitegrid
plt.figure(figsize=(16,5),dpi=150)
plt.subplot(1,2,1)
plt.plot(history['LSTM_test_loss'][1:],label='LSTM')
plt.plot(history['TextCNN_test_loss'][10:],label='TextCNN')
plt.plot(history['CNNText_inception_test_loss'],label='CNNText_inception')
plt.title('Test Loss')
plt.legend(frameon=True,loc='lower right',facecolor='white')
plt.xticks(range(1,21))
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(5)) # 横坐标间隔为5

plt.subplot(1,2,2)
plt.plot(history['LSTM_test_accuracy'][1:],label='LSTM')
plt.plot(history['TextCNN_test_accuracy'][10:],label='TextCNN')
plt.plot(history['CNNText_inception_test_accuracy'],label='CNNText_inception')
plt.title('Test Accuracy')
plt.legend(frameon=True,loc='lower right',facecolor='white')
plt.xticks(range(1,21))
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(5)) # 横坐标间隔为5

fig.show()